# Diagramming


Several diagramming notations have been used for designing relational schema, as already introduced in the section on [Relational Theory](../20-concepts/01-relational.md).

DataJoint comes with a powerful diagramming notation that becomes useful for both data modeling in the design phase and for exploring an existing schema.

A DataJoint schema is depicted as a Directed Acyclic Graph (DAG), where nodes are tables and edges represent the foreign key constraints between them.

Let's start with a simple schema called "projects" depicting employees, projects, and assignments of projects to employees, a many-to-many relationship.




In [1]:
import datajoint as dj

schema = dj.Schema("projects")


@schema
class Employee(dj.Manual):
    definition = """
    employee_id : int
    ---
    employee_name : varchar(60)
    """


@schema
class Project(dj.Manual):
    definition = """
    project_code  : varchar(8)
    ---
    project_title : varchar(50)
    start_date : date
    end_date : date
    """

@schema
class Assignment(dj.Manual):
    definition = """
    -> Employee
    -> Project
    ---
    percent_effort : decimal(4,1) unsigned
    """



[2024-10-21 01:31:00,046][INFO]: Connecting root@localhost:3306
[2024-10-21 01:31:00,065][INFO]: Connected root@localhost:3306
